# Introducing the Clorus

The implementation for `clorus` is very similar to `plip`, so we can simply use `plip`'s template, with a few differences:

1. For actions that use energy, don't set a condition where if the `energy` is negative we set it to 0 instead. This will somehow causes the cloruses to live forever.

In [ ]:
public void move() {
    energy -= 0.03;
}

public void stay() {
    energy -= 0.01;
}

2. For the `attack` method, simply add the clorus's energy by the attacked creature's energy. We don't need to worry about making sure the attacked creature dies since it's already taken care somehow by the simulator.

In [ ]:
public void attack(Creature c) {
    energy += c.energy();
}

3. For `chooseAction()`
* Add a new deque `plipNeighbors` that functions similarly to `emptyNeighbors`, with the only difference that it stores adjacent plips. We need `plipNeighbors` in case a clorus is surrounded by multiple plips, the clorus must randomly pick a plip to attack.

In [ ]:
Deque<Direction> emptyNeighbors = new ArrayDeque<>();
Deque<Direction> plipNeighbors = new ArrayDeque<>();
boolean anyPlip = false;
for (Direction key: neighbors.keySet()) {
    if (neighbors.get(key).name().equals("empty")) {
        emptyNeighbors.addLast(key);
    } else if (neighbors.get(key).name().equals("plip")) {
        plipNeighbors.addLast(key);
        anyPlip = true;
    }
}

Regarding how the rules work, we prioritize the case when an adjacent plip is present.
* If we prioritized the case to STAY when there's no empty neighbor, beware that a plip also takes space!
* This would cause a clorus to do nothing even when a plip is around.

In [ ]:
if (anyPlip) {
    // If a plip is present, attack a plip randomly!
    return new Action(Action.ActionType.ATTACK, randomEntry(plipNeighbors));
} else if (emptyNeighbors.isEmpty()) {
    // If no empty spots (and no plips), just stay
    return new Action(Action.ActionType.STAY);
} else if (energy >= 1) {
    // If energy is greater or equal to 1, then replicate to a random empty spot
    return new Action(Action.ActionType.REPLICATE, randomEntry(emptyNeighbors));
} else {
    // Otherwise, move to a random empty spot
    return new Action(Action.ActionType.MOVE, randomEntry(emptyNeighbors));
}


## Writing Tests

We can basically use the template provided in `TestPLip`, with the only difference that we would need to tweak some elements for testing the `attack()` method.

In [ ]:
// Plip present, attack!
c = new Clorus(1.2);
Plip p = new Plip(.99);
HashMap<Direction, Occupant> topPlip = new HashMap<Direction, Occupant>();
topPlip.put(Direction.TOP, new Plip());
topPlip.put(Direction.BOTTOM, new Empty());
topPlip.put(Direction.LEFT, new Empty());
topPlip.put(Direction.RIGHT, new Empty());

actual = c.chooseAction(topPlip);
expected = new Action(Action.ActionType.ATTACK, Direction.TOP);
assertEquals(expected, actual);
